In [ ]:
import os
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.utils import plot_model
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.layers import Input, Conv2D, Flatten, Dense,Concatenate, Conv2DTranspose, Reshape, Lambda, Activation, BatchNormalization, LeakyReLU, Dropout, ZeroPadding2D, UpSampling2D
from tensorflow.keras.models import Model, Sequential, load_model
from tensorflow.keras import backend as K
from tensorflow.keras.initializers import RandomNormal

import math
import matplotlib.pyplot as plt

import pypianoroll
import numpy as np
from pypianoroll import Multitrack, Track

In [ ]:
# run params
run_id = '3001'
music_name = 'gan/'

RUN_FOLDER = 'run/'
RUN_FOLDER += '_'.join([run_id, music_name])
IMAGE_FOLDER = os.path.join(RUN_FOLDER, "test_extreme/")
# Number of timestept the slices Pianorolls should have (Needs to be dividable by 16)
pianrollLength = 128

store_folder = os.path.join(RUN_FOLDER, 'store')
data_folder = os.path.join('data', music_name)

if not os.path.exists(RUN_FOLDER):
    os.makedirs(RUN_FOLDER)
    os.mkdir(os.path.join(RUN_FOLDER, 'store'))
    os.mkdir(os.path.join(RUN_FOLDER, 'output'))
    os.mkdir(os.path.join(RUN_FOLDER, 'weights'))
    os.mkdir(os.path.join(RUN_FOLDER, 'viz'))

if not os.path.exists(IMAGE_FOLDER):
    os.mkdir(IMAGE_FOLDER)   

#weight_init = RandomNormal(mean=0., stddev=0.02)
weight_init = tf.keras.initializers.GlorotUniform(seed=None)
#weight_init = tf.keras.initializers.he_uniform(seed=None)

In [ ]:
# Set Note bounds for faster training
lowestNotePossible = 20
highestNotePossible = 108
# possibleNotes mus be dividable by 4 else the Architekture needs to be changed
possibleNotes = highestNotePossible - lowestNotePossible

In [ ]:
# Save the numpyArray for further us
'''
reshaped = np.load('data/preprocessed/midi_p128_dn88.npy')
isNormalized = False

reshaped = np.load('data/preprocessed/midi_normalized_p128_dn88.npy')
isNormalized = True
'''

reshaped = np.load('data/preprocessed/midi_binarized_p128_dn88.npy')
isNormalized = True

# Creating the neural Network

In [ ]:
### THE discriminator
discriminator_input = Input(shape=(pianrollLength,possibleNotes,1), name='discriminator_input')

x = discriminator_input


x = Conv2D(filters = 64, kernel_size = (3,3), strides=2, kernel_initializer = weight_init)(x)
x = LeakyReLU()(x)
#x = Activation('relu')(x)
x = Dropout(0.1)(x)

x = Conv2D(filters = 64, kernel_size = (6,6), strides=2, kernel_initializer = weight_init)(x)
x = LeakyReLU()(x)
#x = Activation('relu')(x)
x = Dropout(0.1)(x)


x = Conv2D(filters = 128, kernel_size = (6,6), strides=2, kernel_initializer = weight_init)(x)
x = LeakyReLU()(x)
#x = Activation('relu')(x)
x = Dropout(0.1)(x)

x = Conv2D(filters = 256, kernel_size = (6,1), strides=2, kernel_initializer = weight_init)(x)
x = LeakyReLU()(x)
#x = Activation('relu')(x)
x = Dropout(0.1)(x)

x = Flatten()(x)

discriminator_output = Dense(1, activation='sigmoid', kernel_initializer = weight_init)(x)
discriminator = Model(discriminator_input, discriminator_output, name= 'discriminator')

In [ ]:
discriminator.summary()

In [ ]:
# Quelle: https://stackoverflow.com/a/45199301
# Save the summary to a file 
#from contextlib import redirect_stdout

#with open(os.path.join(store_folder, 'modelsummarydiscriminator.txt'), 'w') as f:
#    with redirect_stdout(f):
#        discriminator.summary()

In [ ]:
z_dim = 50

generator_input = Input(shape=(z_dim,), name='generator_input')
generator_initial_dense_layer_size = (int(pianrollLength/4),int(possibleNotes/8),4)

x = generator_input
x = Dense(np.prod(generator_initial_dense_layer_size), kernel_initializer = weight_init)(x)
#x = BatchNormalization(momentum=0.9)(x)

#x = LeakyReLU()(x)
x = Activation('relu')(x)
x = Dropout(0.2)(x)
x = Reshape(generator_initial_dense_layer_size)(x)

### Meldoy ###
m = UpSampling2D(size=(2, 2), data_format=None, interpolation='nearest')(x)
m = Conv2D(filters = 64, kernel_size = (6,6), padding='same', kernel_initializer = weight_init, name = 'generator_melody_conv_0')(m)
#m = Activation('relu')(m)
m = LeakyReLU()(m)
m = Dropout(0.2)(m)

m = UpSampling2D(size=(2, 2), data_format=None, interpolation='nearest')(m)
m = Conv2D(filters = 64, kernel_size = (13,13), padding='same', kernel_initializer = weight_init, name = 'generator_melody_conv_1')(m)
#m = Activation('relu')(m)
m = LeakyReLU()(m)
m = Dropout(0.2)(m)

m = Conv2D(filters = 16, kernel_size = (1,13), padding='same', kernel_initializer = weight_init, name = 'generator_melody_conv_special')(m)
#m = Activation('relu')(m)
m = LeakyReLU()(m)
m = Dropout(0.2)(m)

m = Conv2D(filters = 1, kernel_size = (6,6), padding = 'same', kernel_initializer = weight_init)(m)        
m = Activation('sigmoid')(m)
##############

### Bass Line ####
b = UpSampling2D(size=(2, 2), data_format=None, interpolation='nearest')(x)
b = Conv2D(filters = 64, kernel_size = (6,6), padding='same', kernel_initializer = weight_init, name = 'generator_bass_conv_0')(b)
#b = Activation('relu')(b)
b = LeakyReLU()(b)
b = Dropout(0.2)(b)

b = UpSampling2D(size=(2, 2), data_format=None, interpolation='nearest')(b)
b = Conv2D(filters = 64, kernel_size = (13,13), padding='same', kernel_initializer = weight_init, name = 'generator_bass_conv_1')(b)
b = LeakyReLU()(b)
b = Dropout(0.2)(b)

b = Conv2D(filters = 16, kernel_size = (1,13), padding='same', kernel_initializer = weight_init, name = 'generator_bass_conv_special')(b)
#b = Activation('relu')(b)
b = LeakyReLU()(b)
b = Dropout(0.2)(b)

b = Conv2D(filters = 1, kernel_size = (6,6), padding = 'same', kernel_initializer = weight_init)(b)        
b = Activation('sigmoid')(b)
##################

x = Concatenate(axis=2)([m,b])

#x = Conv2D(filters = 64, kernel_size = (5,5), padding='same', kernel_initializer = weight_init, name = 'generator_conv_1')(x)
#x = Activation('relu')(x)

#x = Conv2D(filters = 1, kernel_size = (6,6), padding = 'same', kernel_initializer = weight_init)(x)        
#x = Activation('sigmoid')(x)


generator_output = x
generator = Model(generator_input, generator_output, name='generator')

In [ ]:
generator.summary()

In [ ]:
# Quelle: https://stackoverflow.com/a/45199301
# Save the summary to a file 
#from contextlib import redirect_stdout

#with open(os.path.join(store_folder, 'modelsummarygenerator.txt'), 'w') as f:
#    with redirect_stdout(f):
#        generator.summary()

In [ ]:
def set_trainable(model, isTrainable):
    for layer in model.layers:
        layer.trainable = isTrainable

In [ ]:
### COMPILE DISCRIMINATOR
discriminator.compile(
optimizer=RMSprop(lr=0.0004)
, loss = 'binary_crossentropy'
,  metrics = ['accuracy']
)
        
### COMPILE THE FULL GAN
set_trainable(discriminator, False)

model_input = Input(shape=(z_dim,), name='model_input')
model_output = discriminator(generator(model_input))
GANModel = Model(model_input, model_output)

opti = RMSprop(learning_rate=0.0001)
GANModel.compile(optimizer=opti , loss='binary_crossentropy', metrics=['accuracy'])

set_trainable(discriminator, True)


In [ ]:
#plot_model(model, to_file=os.path.join(RUN_FOLDER ,'viz/model.png'), show_shapes = True, show_layer_names = True)
#plot_model(discriminator, to_file=os.path.join(RUN_FOLDER ,'viz/discriminator.png'), show_shapes = True, show_layer_names = True)
#plot_model(generator, to_file=os.path.join(RUN_FOLDER ,'viz/generator.png'), show_shapes = True, show_layer_names = True)

# Training of the GAN

In [ ]:
def generate_Midi_File(img, epoch, isNormalized):
    gen_img = img.reshape(pianrollLength, 88)
    result = np.zeros((pianrollLength, 128))
    result[:,lowestNotePossible:highestNotePossible] = gen_img
    #if isNormalized:
    #result = (result > 0.1) * 255.
    result = result * 255.
    
    track = Track(pianoroll=result, program=0, is_drum=False,name='my awesome piano')

    multi = Multitrack()
    multi.tracks[0] = track
    pypianoroll.write(multi, os.path.join(IMAGE_FOLDER, str(epoch)+".mid"))
    
    multi.tracks[0].plot()
    #fig.set_size_inches(100,100)
    #plt.show()
    plt.savefig(os.path.join(IMAGE_FOLDER, str(epoch)+"_pianoroll.png"), format='png')
    plt.close()

In [ ]:
def sample_images(epoch):
        r, c = 5, 5
        noise = np.random.normal(0, 1, (r * c, z_dim))
        gen_imgs = generator.predict(noise)

        #Rescale images 0 - 1

        fig, axs = plt.subplots(r, c, figsize=(5,15))
        cnt = 0

        for i in range(r):
            for j in range(c):
                axs[i,j].imshow(np.squeeze(gen_imgs[cnt, :,:,:]), cmap = 'gray')
                axs[i,j].axis('off')
                cnt += 1
        fig.savefig(os.path.join(IMAGE_FOLDER, "%d_multi.png" % epoch))
        plt.close()

In [ ]:
def train_normal(epochs, batch_size=32):
    # Save the Generator and discriminator models
    #save_model(generator, os.path.join(RUN_FOLDER, 'images/generator'))
    #save_model(discriminator, os.path.join(RUN_FOLDER, 'images/discriminator'))
    #save_model(GANModel, os.path.join(RUN_FOLDER, r"images/GANModel"))
    # Adversarial ground truths
    valid = np.ones((batch_size, 1))
    fake = np.zeros((batch_size, 1))
    # Label Smoothening
    valid_smoothened = np.random.uniform(low=0.8, high=1.0, size=(batch_size,1))
    fake_smoothened = np.random.uniform(low=0.0, high=0.05, size=(batch_size,1))
    
    for epoch in range(epochs):

        # ---------------------
        #  Train Discriminator
        # ---------------------      
        noise = np.random.normal(0, 1, (batch_size, z_dim))
        gen_imgs = generator.predict(noise)
        # Select a random half of images
        idx = np.random.randint(0, reshaped.shape[0], batch_size)
        imgs = reshaped[idx]
        
        # ---------------------
        #  Train the discriminator (real classified as ones and generated as zeros)
        # ---------------------
        d_loss_real, d_acc_real =  discriminator.train_on_batch(imgs, valid)
        d_loss_fake, d_acc_fake =  discriminator.train_on_batch(gen_imgs, fake)
        d_loss =  0.5 * (d_loss_real + d_loss_fake)
        d_acc = 0.5 * (d_acc_real + d_acc_fake)

        # ---------------------
        #  Train Generator
        # ---------------------
        g_loss = GANModel.train_on_batch(noise, valid)
        #print ("%d [G loss: %.3f] [G acc: %.3f]" % (epoch, g_loss[0], g_loss[1]))
        
        # ---------------------
        #  Save Losses for evaluation
        # ---------------------
        d = [d_loss, d_loss_real, d_loss_fake, d_acc, d_acc_real, d_acc_fake]
        d_losses.append(d)
        g = [g_loss[0], g_loss[1]]
        g_losses.append(g)
        
        
        if (epoch % 100 == 0): 
            # Save an example
            fig=plt.figure(figsize=(16, 64))
            plt.imshow(gen_imgs[0, :, :, 0], cmap='gray')
            plt.axis('off')
            plt.savefig(os.path.join(IMAGE_FOLDER, str(epoch)+".png"), format='png')
            generate_Midi_File(gen_imgs[0, :, :, 0], epoch, isNormalized)
            plt.close()
            
            # Save some examples
            sample_images(epoch)
            
            # Continuiously save a plot with the new values to see the development of the loss
            fig = plt.figure()
            plt.plot([x[0] for x in d_losses], color='black', linewidth=0.25)
            plt.plot([x[1] for x in d_losses], color='green', linewidth=0.25)
            plt.plot([x[2] for x in d_losses], color='red', linewidth=0.25)
            plt.plot([x[0] for x in g_losses], color='orange', linewidth=0.25)

            plt.xlabel('batch', fontsize=18)
            plt.ylabel('loss', fontsize=16)

            # plt.xlim(0, 2000)
            #plt.ylim(0, 50)

            plt.savefig(os.path.join(IMAGE_FOLDER, "loss_chart.png"), format='png')
            plt.show()
            plt.close
            # Save the loss arrays
            np.save(os.path.join(IMAGE_FOLDER, "D_loss.npy"), d_losses)
            np.save(os.path.join(IMAGE_FOLDER, "G_loss.npy"), g_losses)
            
            if (epoch % 1000 == 0):
                GANModel.save(os.path.join(IMAGE_FOLDER, 'GANModel_'+str(epoch)+'_loss_'+str(g_loss[0])+'.h5'))
                discriminator.save(os.path.join(IMAGE_FOLDER, 'discriminator_'+str(epoch)+'_loss_'+str(d_loss)+'.h5'))
                generator.save(os.path.join(IMAGE_FOLDER, 'generator_'+str(epoch)+'_loss_'+str(g_loss[0])+'.h5'))
                
                
        # Plot the progress
        if (epoch % 10 == 0):
            print ("%d [D loss: (%.3f)(R %.3f, F %.3f)] [D acc: (%.3f)(%.3f, %.3f)] [G loss: %.3f] [G acc: %.3f]" % (epoch, d_loss, d_loss_real, d_loss_fake, d_acc, d_acc_real, d_acc_fake, g_loss[0], g_loss[1]))

In [ ]:
d_losses = []
g_losses = []

train_normal(100001, batch_size=64)

In [ ]:
fig = plt.figure()
plt.plot([x[0] for x in d_losses], color='black', linewidth=0.25)
plt.plot([x[1] for x in d_losses], color='green', linewidth=0.25)
plt.plot([x[2] for x in d_losses], color='red', linewidth=0.25)
plt.plot([x[0] for x in g_losses], color='orange', linewidth=0.25)

plt.xlabel('batch', fontsize=18)
plt.ylabel('loss', fontsize=16)

#plt.xlim(0, 20000)
plt.ylim(0, 30)

plt.savefig(os.path.join(IMAGE_FOLDER, "loss_chart2.png"), format='png')
plt.show()
plt.close

In [ ]:
# Quelle: https://stackoverflow.com/a/6537563
# Beep to tell training finished
import winsound
frequency = 300  # Set Frequency To 2500 Hertz
duration = 1000  # Set Duration To 1000 ms == 1 second
winsound.Beep(frequency, duration)

In [ ]:
#d_losses = np.load(os.path.join(IMAGE_FOLDER, "D_loss.npy"))
#g_losses = np.load(os.path.join(IMAGE_FOLDER, "G_loss.npy"))

In [ ]:
#generator = load_model(os.path.join(IMAGE_FOLDER, "First_run/generator_10000_loss_30.628813.h5"))
#discriminator = load_model(os.path.join(IMAGE_FOLDER, "First_run/discriminator_10000_loss_5.734544512425027e-14.h5"))
#GANModel = load_model(os.path.join(IMAGE_FOLDER, "First_run/GANModel_10000_loss_30.628813.h5"))


In [ ]:
#generator.summary()